# Transactions with absolute locktime - BX Exercise
In this BX tutorial, we will send funds to a `checklocktimeverify` encumbered output, and spend this after the absolute locktime has expired.
<img src="images/cltv_overview.jpg" alt="drawing" style="" width="700px"/>

<hr style="border: 0.5px dashed #000;">

## 1. Restore wallet and generate spending key pairs.


In [1]:
# my_mnemonic ="word0 word1 word2 ..."
# hd_master_private=$( )
# hd_master_public=$( )

my_mnemonic="seven mail crash you unit small assume express wedding cloud work potato"
hd_master_private=$(bx mnemonic-to-seed $my_mnemonic | bx hd-new)
hd_master_public=$(bx hd-to-public $hd_master_private)

### Restore your spending keys.

* `m/44'/1'/1'/0/4`

In [2]:
# hd_m_44h_1h_1h_0_4=

# privatekey_44h_1h_1h_0_4=
# publickey_44h_1h_1h_0_4=
# publickeyhash_44h_1h_1h_0_4=
# address_44h_1h_1h_0_4=

hd_m_44h_1h_1h_0_4=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 1  \
| bx hd-private --index 0 \
| bx hd-private --index 4)

privatekey_44h_1h_1h_0_4=$(bx hd-to-ec $hd_m_44h_1h_1h_0_4)
publickey_44h_1h_1h_0_4=$(bx hd-to-public $hd_m_44h_1h_1h_0_4 | bx hd-to-ec)
publickeyhash_44h_1h_1h_0_4=$(bx sha256 $publickey_44h_1h_1h_0_4 | bx ripemd160)
address_44h_1h_1h_0_4=$(bx hd-to-public $hd_m_44h_1h_1h_0_4 | bx hd-to-ec | bx ec-to-address --version 111)

echo $address_44h_1h_1h_0_4
echo $publickeyhash_44h_1h_1h_0_4

mm2bGEdHUeZr6Yqx5b1rnuqi7cqTbjK9hG
3c738d3b4af3572fa74f02ae488a99374c0726ad


### Generate destination keys to send funds to.

* `m/44'/1'/2'/0/4`

In [3]:
# hd_m_44h_1h_2h_0_4=

# privatekey_44h_1h_2h_0_4=
# publickey_44h_1h_2h_0_4=
# publickeyhash_44h_1h_2h_0_4=
# address_44h_1h_2h_0_4=

hd_m_44h_1h_2h_0_4=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 2  \
| bx hd-private --index 0 \
| bx hd-private --index 4)

privatekey_44h_1h_2h_0_4=$(bx hd-to-ec $hd_m_44h_1h_2h_0_4)
publickey_44h_1h_2h_0_4=$(bx hd-to-public $hd_m_44h_1h_2h_0_4 | bx hd-to-ec)
publickeyhash_44h_1h_2h_0_4=$(bx sha256 $publickey_44h_1h_2h_0_4 | bx ripemd160)
address_44h_1h_2h_0_4=$(bx hd-to-public $hd_m_44h_1h_2h_0_4 | bx hd-to-ec | bx ec-to-address --version 111)

echo $address_44h_1h_2h_0_4
echo $publickeyhash_44h_1h_2h_0_4

my9EknDkaPJio8RRhaQVF52jCwEWaDAtG3
c156da8e421222be6665c78878ea49220d2ecb52


### Fetch previous UTXO's you are spending.

In [4]:
# bx fetch-tx [transaction_hash] -f json | jq ".transaction.outputs[output_index]"

bx fetch-tx 11a8587f5077fc7f69e3349797fd74f99864d27716d6b48ac51734cb3bec03ea -f json | jq ".transaction.outputs[4]"


{
  "address_hash": "3c738d3b4af3572fa74f02ae488a99374c0726ad",
  "script": "dup hash160 [3c738d3b4af3572fa74f02ae488a99374c0726ad] equalverify checksig",
  "value": "499550"
}


In [5]:
# previous_txid=
# previous_output_index=
# previous_output_amount=

previous_txid=11a8587f5077fc7f69e3349797fd74f99864d27716d6b48ac51734cb3bec03ea
previous_output_index=4
previous_output_amount=499550

<hr style="border: 0.5px dashed #000;">

## 2) Spending to a CLTV encumbered output<br>
<img src="images/spend_to_cltv.jpg" alt="drawing" style="" width="700px"/>

### A. Build transaction with CLTV encumbered output for signing

#### Determine absolute locktime in blockheight
* `$ bx fetch-height`
* `$ expr [current blockheight] + [delay in blocks]`.
* Encode locktime in hex (little endian)

In [6]:
delay_decimal=4
locktime_decimal=$(expr $(bx fetch-height) + $delay_decimal)
echo $locktime_decimal


1442785


In [7]:
printf '%x\n' $locktime_decimal

1603e1


**Note:** Don't forget to flip byte-order (4-Bytes) above for output script below.

#### Encode CLTV-encumbered P2PKH output script
* **`[locktime]`**
* **`CHECKLOCKTIMEVERIFY`**
* `DROP`
* `DUP`
* `HASH160`
* `[public key hash]`
* `EQUALVERIFY`
* `CHECKSIG`

In [8]:
# bx script-encode "[time] CHECKLOCKTIMEVERIFY DUP HASH160 [public key hash] EQUALVERIFY CHECKSIG"

# output_script=

output_script=$(bx script-encode "[e10316] CHECKLOCKTIMEVERIFY DROP DUP HASH160 ["$publickeyhash_44h_1h_2h_0_4"] EQUALVERIFY CHECKSIG")
echo $output_script

03e10316b17576a914c156da8e421222be6665c78878ea49220d2ecb5288ac


#### Calculate output amount

* `400 Satoshi/sigops + 1 Satoshi/Byte` 
* Output amount(s) are previous output amount(s) minus fees

In [31]:
# tx bytes: 4 + 1 + 1*(32+4+1+1+72+1+33+4) + 1 + 1*(8+1+(6+25)) + 4 

# tx_byte_count=

tx_byte_count=$(expr 4 + 1 + 1 \* 148 + 1 + 1 \* 40 + 4)
echo $tx_byte_count


198


In [32]:
# Output amount total.

# output_amount=

output_amount=$(expr $previous_output_amount - 1 \* 400 - $tx_byte_count \* 1)
echo $output_amount


498952


#### Encode transaction for signing
* `bx tx-encode -i [prevout txid:index:sequence] -o [output script:amount]`
* Input sequence defaults to `ffffffff`/`4294967295`

In [11]:
# bx tx-encode \
# --input [previous tx id]:[index]:[sequence] \
# --output [output script]:[output amount]

# cltv_tx=

cltv_tx=$(bx tx-encode \
--input $previous_txid:$previous_output_index:4294967295 \
--output $output_script:$output_amount)
echo $cltv_tx


0100000001ea03ec3bcb3417c58ab4d61677d26498f974fd979734e3697ffc77507f58a8110400000000ffffffff01089d0700000000001f03e10316b17576a914c156da8e421222be6665c78878ea49220d2ecb5288ac00000000


### Sign the transaction

**Fetch previous output script**

In [30]:
# bx fetch-tx --format json [previous tx id] 

# previous_output_script=

previous_output_script=$(bx fetch-tx --format json $previous_txid \
| jq -r ".transaction.outputs[4].script")
echo $previous_output_script


dup hash160 [3c738d3b4af3572fa74f02ae488a99374c0726ad] equalverify checksig


**Sign transaction with previous output script.**

In [13]:
# bx input-sign [private key] "previous output script" [transaction template]

# signature= 

signature=$(bx input-sign --sign_type all --index 0 $privatekey_44h_1h_1h_0_4 "$previous_output_script" $cltv_tx)


**Set the input script into the finalised transaction.**

In [14]:
# bx input-set "[signature] [public key point]" [transaction template]

# cltv_tx=

cltv_tx=$(bx input-set --index 0 "[$signature] [$publickey_44h_1h_1h_0_4]" $cltv_tx)


In [15]:
# Validate.
bx validate-tx $cltv_tx


matching transaction with unspent outputs


: 255

In [16]:
# Decode
bx tx-decode -f json $cltv_tx |jq

{
  "transaction": {
    "hash": "a4108a6d3de5db855fff07935807c0526db1f070542103415469faea335172de",
    "inputs": [
      {
        "address_hash": "3c738d3b4af3572fa74f02ae488a99374c0726ad",
        "previous_output": {
          "hash": "11a8587f5077fc7f69e3349797fd74f99864d27716d6b48ac51734cb3bec03ea",
          "index": "4"
        },
        "script": "[3045022100ecc29b6e4ccbb04bc077bfd6ae6d2a49dd04a7f53c930c017878debb5983c08602201acba5f6487477472a26a8b2e91245d54e84c7c852a2fb742c4df6f90914c46201] [03b8de3d8d5855936b27d99825cfb7913a7982252e4fbd749c3d1c9b94af28908f]",
        "sequence": "4294967295"
      }
    ],
    "lock_time": "0",
    "outputs": [
      {
        "script": "[e10316] checklocktimeverify drop dup hash160 [c156da8e421222be6665c78878ea49220d2ecb52] equalverify checksig",
        "value": "498952"
      }
    ],
    "version": "1"
  }
}


In [27]:
# Send
bx send-tx $cltv_tx


matching transaction with unspent outputs


: 255

<hr style="border: 0.5px dashed #000;">

## 3) Spend the CLTV encumbered UXTO

<img src="images/spend_from_cltv.jpg" alt="drawing" style="" width="700px"/>

### A. Build p2pkh output.


#### Encode p2pkh output script

* Please send testnet coins to the following address: `n2MBcctgzBt1h8Nvfu3XAEPJLrmWET7emw`

In [17]:
bx address-decode -f json n2MBcctgzBt1h8Nvfu3XAEPJLrmWET7emw | jq ".wrapper.payload"


"e48199d47742b245464b1366d95ef26aa4c8bb2c"


In [18]:
# output_script=

output_script=$(bx script-encode "DUP HASH160 [e48199d47742b245464b1366d95ef26aa4c8bb2c] EQUALVERIFY CHECKSIG")
echo $output_script


76a914e48199d47742b245464b1366d95ef26aa4c8bb2c88ac


**Encode transaction with locktime & set input sequence**<br>
* Reference Previous Outpoint<br>
* Transaction locktime field must be `>=` locktime in CLTV output being spent.
* Input sequence must be less than maximum, so `fffffffe`/`4294967294`

In [19]:
# tx bytes: 4 + 1 + 1*(32+4+1+1+72+1+33+4) + 1 + 1*(8+1+25) + 4 

# tx_byte_count=

tx_byte_count=$(expr 4 + 1 + 1 \* 148 + 1 + 1 \* 34 + 4)
echo $tx_byte_count


192


In [29]:
# Output amount total.

# output_amount=

output_amount=$(expr $previous_output_amount - 1 \* 400 - $tx_byte_count \* 1)
echo $output_amount


498958


In [21]:
# bx tx-encode \
# --input [previous tx id]:[index]:[sequence] \
# --output [output script]:[output amount]

# my_tx=

my_tx=$(bx tx-encode \
--lock_time $locktime_decimal \
--input $previous_txid:$previous_output_index:4294967294 \
--output $output_script:$output_amount)


### Sign and set the input script

#### Signing the transaction with the private key

In [22]:
# cltv_tx_txid=

cltv_tx_txid=$(bx bitcoin256 $cltv_tx)
echo $cltv_tx_txid


a4108a6d3de5db855fff07935807c0526db1f070542103415469faea335172de


In [23]:
# previous_output_script=

previous_output_script=$(bx fetch-tx --format json $cltv_tx_txid \
| jq -r ".transaction.outputs[0].script")
echo $previous_output_script


[e10316] checklocktimeverify drop dup hash160 [c156da8e421222be6665c78878ea49220d2ecb52] equalverify checksig


**Sign transaction with previous output script.**

In [24]:
signature=$(bx input-sign --sign_type all --index 0 $privatekey_44h_1h_2h_0_4 "$previous_output_script" $my_tx)


**Set the input script into the finalised transaction.**

In [25]:
my_tx=$(bx input-set --index 0 "[$signature] [$publickey_44h_1h_2h_0_4]" $my_tx)


In [26]:
# Validate.
bx validate-tx $my_tx


transaction currently non-final for next block


: 255

In [ ]:
# Send.
